In [1]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Imports de la libreria propia
from vecopsciml.kernels.derivative import DerivativeKernels
from vecopsciml.utils import TensOps

# Imports de las funciones creadas para este programa
from models.non_constant_diffusivity import NonConstantDiffusivityNeuralNetwork
from utils.folders import create_folder
from utils.load_data import load_data
from trainers.train import train_loop

In [2]:
# Creamos los paths para las distintas carpetas
ROOT_PATH = r'C:\Users\usuario\Desktop\rmunozTMELab\Physically-Guided-Machine-Learning'
DATA_PATH = os.path.join(ROOT_PATH, r'data\non_linear\non_linear_fft_tests.pkl')
RESULTS_FOLDER_PATH = os.path.join(ROOT_PATH, r'results\non_linear')
MODEL_RESULTS_PATH = os.path.join(ROOT_PATH, r'results\non_linear\model_fft')

# Creamos las carpetas que sean necesarias (si ya están creadas se avisará de ello)
create_folder(RESULTS_FOLDER_PATH)
create_folder(MODEL_RESULTS_PATH)

Folder already exists at: C:\Users\usuario\Desktop\rmunozTMELab\Physically-Guided-Machine-Learning\results\non_linear
Folder already exists at: C:\Users\usuario\Desktop\rmunozTMELab\Physically-Guided-Machine-Learning\results\non_linear\model_fft


In [3]:
# Load dataset
dataset = load_data(DATA_PATH)

Data successfully loaded from: C:\Users\usuario\Desktop\rmunozTMELab\Physically-Guided-Machine-Learning\data\non_linear\non_linear_fft_tests.pkl


In [4]:
# Convolutional filters to derivate
dx = dataset['x_step_size']
dy = dataset['y_step_size']
D = DerivativeKernels(dx, dy, 0).grad_kernels_two_dimensions()

In [5]:
u = torch.Tensor(dataset['y_train'])

f_coef = torch.fft.fft2(u)
F_ordered = f_coef.flatten(start_dim=1)

mask = np.zeros_like(F_ordered)
mask[:, 0:10] = 1

F_full = f_coef*mask.reshape(f_coef.shape)
u_reconstructed = torch.fft.ifft2(F_full).real

# y_train = F_ordered[:, 0:10].reshape(u.shape[0], 10)

real_part = F_ordered[:, 0:10].reshape(u.shape[0], 10).real
imag_part = F_ordered[:, 0:10].reshape(u.shape[0], 10).imag

y_train = torch.stack([
    real_part,
    imag_part,
], dim=-1)

In [6]:
X_train = torch.Tensor(dataset['X_train']).unsqueeze(1)
K_train = torch.tensor(dataset['k_train'], dtype=torch.float32, requires_grad=True)
f_train = torch.tensor(dataset['f_train'], dtype=torch.float32, requires_grad=True)

input_size = X_train[0].shape
predictive_output_size = y_train[0].shape
explanatory_output_size = K_train[0].shape

In [7]:
import torch
import torch.nn as nn

from vecopsciml.utils import TensOps
from vecopsciml.operators.zero_order import Mx, My

class FFTNeuralNetwork(nn.Module):

    def __init__(self, input_size, predictive_output_size, explanatory_output_size, **kwargs):
        super(FFTNeuralNetwork, self).__init__()

        self.input = input_size
        self.output_pred = predictive_output_size
        self.output_expl = explanatory_output_size

        self.hidden_units_pred = 10
        self.hidden_units_exp = 15
        self.filters_exp = 10

        # Predictive network
        self.flatten_layer_pred = nn.Flatten(start_dim=1, end_dim=-1)
        self.hidden1_layer_pred = nn.Linear(torch.prod(torch.tensor(self.input)), self.hidden_units_pred)  
        self.hidden2_layer_pred = nn.Linear(self.hidden_units_pred, self.hidden_units_pred)  
        self.output_layer_pred = nn.Linear(self.hidden_units_pred, self.output_pred[0]*self.output_pred[1])

        # Explanatory network
        self.conv1_exp = nn.Conv2d(in_channels=1, out_channels=self.filters_exp, kernel_size=1)
        self.flatten_layer_exp = nn.Flatten()
        self.hidden1_layer_exp = nn.LazyLinear(self.hidden_units_exp)
        self.hidden2_layer_exp = nn.Linear(self.hidden_units_exp, self.hidden_units_exp)
        self.output_layer_exp = nn.Linear(self.hidden_units_exp, self.filters_exp * (self.output_expl[0] - 1) * (self.output_expl[1] - 1))
        self.conv2_exp = nn.Conv2d(in_channels=self.filters_exp, out_channels=1, kernel_size=1)


    def forward(self, X):

        # Predictive network
        X = self.flatten_layer_pred(X)
        X = torch.sigmoid(self.hidden1_layer_pred(X))
        X = torch.sigmoid(self.hidden2_layer_pred(X))
        output_predictive_net = self.output_layer_pred(X)

        # Obtain real and imaginary part of output and transform it in a complex number
        output_predictive = output_predictive_net.view(output_predictive_net.size(0), self.output_pred[0], self.output_pred[1])
        real = output_predictive[..., 0]
        imag = output_predictive[..., 1]
        
        # Reconstruction of u(x, y) with iFFT
        f_coef = torch.zeros(output_predictive_net.size(0), 1, self.output_expl[0], self.output_expl[1], dtype=torch.complex64)
        f_coef[:, 0, 0:10, 0] = torch.complex(real, imag)

        u_pred = torch.fft.ifft2(f_coef).real
        um_pred = Mx(My(TensOps(u_pred, space_dimension=2, contravariance=0, covariance=0))).values

        # Explanatory network
        x = torch.sigmoid(self.conv1_exp(um_pred))
        x = self.flatten_layer_exp(x)
        x = torch.sigmoid(self.hidden1_layer_exp(x))
        x = torch.sigmoid(self.hidden2_layer_exp(x))
        x = self.output_layer_exp(x)
        x = x.view(x.size(0), self.filters_exp, self.output_expl[0] - 1, self.output_expl[1] - 1)
        K_pred = self.conv2_exp(x)

        return output_predictive, u_pred, K_pred

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")

Using device: cpu


In [9]:
def fourier_transform_split(dataset):

    u = torch.Tensor(dataset)
    
    f_coef = torch.fft.fft2(u)
    
    F_ordered = f_coef.flatten(start_dim=1)
    
    mask = np.zeros_like(F_ordered)
    mask[:, 0:10] = 1
    
    F_full = f_coef * mask.reshape(f_coef.shape)
    
    u_reconstructed = torch.fft.ifft2(F_full).real
    
    real_part = F_ordered[:, 0:10].reshape(u.shape[0], 10).real
    imag_part = F_ordered[:, 0:10].reshape(u.shape[0], 10).imag
    
    y_transformed = torch.stack([real_part, imag_part], dim=-1)
    
    return y_transformed

In [10]:
# Tratamiento de los datos para dividirlos en train y test
X_train = torch.Tensor(dataset['X_train']).unsqueeze(1)
y_train = TensOps(fourier_transform_split(dataset['y_train']).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
K_train = TensOps(torch.tensor(dataset['k_train'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
f_train = TensOps(torch.tensor(dataset['f_train'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)

X_val = torch.Tensor(dataset['X_val']).unsqueeze(1)
y_val = TensOps(fourier_transform_split(dataset['y_val']).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
K_val = TensOps(torch.tensor(dataset['k_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
f_val = TensOps(torch.tensor(dataset['f_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)

X_np = X_train
y_np = y_train.values
K_np = K_train.values
f_np = f_train.values

X_train_np, X_test_np, y_train_np, y_test_np, K_train_np, K_test_np, f_train_np, f_test_np = train_test_split(X_np, y_np, K_np, f_np, test_size=0.2, random_state=42)

X_train = X_train_np.to(device)
X_test = X_test_np.to(device)

y_train = TensOps(y_train_np.to(device), space_dimension=y_train.space_dim, contravariance=0, covariance=0)
y_test = TensOps(y_test_np.to(device), space_dimension=y_train.space_dim, contravariance=0, covariance=0)

K_train = TensOps(K_train_np.to(device), space_dimension=K_train.space_dim, contravariance=0, covariance=0)
K_test = TensOps(K_test_np.to(device), space_dimension=K_train.space_dim, contravariance=0, covariance=0)

f_train = TensOps(f_train_np.to(device), space_dimension=K_train.space_dim, contravariance=0, covariance=0)
f_test = TensOps(f_test_np.to(device), space_dimension=K_train.space_dim, contravariance=0, covariance=0)

In [11]:
# Se carga el modelo y el optimizador
model = FFTNeuralNetwork(input_size, predictive_output_size, explanatory_output_size)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

# Parametros de entrenamiento
start_epoch = 0
n_epochs = 20000

batch_size = 64
n_checkpoints = 100

train_loop(model, optimizer, n_checkpoints,
           X_train, y_train, X_test, y_test, f_train, f_test,
           D=D, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PATH, device=device,
        )

Start training
Epoch 0, Train loss: 1.039e+12, Test loss: 1.002e+12, MSE(e): 1.039e+04, MSE(pi1): 4.494e+00, MSE(pi2): 4.174e+01, MSE(pi3): 2.754e-01
Epoch 100, Train loss: 9.614e+11, Test loss: 9.275e+11, MSE(e): 9.614e+03, MSE(pi1): 6.128e-01, MSE(pi2): 3.926e+01, MSE(pi3): 2.231e-01
Epoch 200, Train loss: 8.863e+11, Test loss: 8.548e+11, MSE(e): 8.863e+03, MSE(pi1): 6.045e-01, MSE(pi2): 3.609e+01, MSE(pi3): 2.161e-01
Epoch 300, Train loss: 8.232e+11, Test loss: 7.936e+11, MSE(e): 8.232e+03, MSE(pi1): 6.039e-01, MSE(pi2): 3.353e+01, MSE(pi3): 2.162e-01
Epoch 400, Train loss: 7.658e+11, Test loss: 7.379e+11, MSE(e): 7.658e+03, MSE(pi1): 6.040e-01, MSE(pi2): 3.124e+01, MSE(pi3): 2.163e-01
Epoch 500, Train loss: 7.116e+11, Test loss: 6.853e+11, MSE(e): 7.116e+03, MSE(pi1): 6.040e-01, MSE(pi2): 2.909e+01, MSE(pi3): 2.163e-01
Epoch 600, Train loss: 6.600e+11, Test loss: 6.353e+11, MSE(e): 6.600e+03, MSE(pi1): 6.041e-01, MSE(pi2): 2.703e+01, MSE(pi3): 2.163e-01
Epoch 700, Train loss: 6.105